# Reconhecimento de Padrões
#### UTFPR — Universidade Tecnológica Federal do Paraná 
#### Nomes: Rafael Menezes Barboza, Carlos Alexandre Peron dos Santos


Este trabalho tem como objetivo buscar por caracteristicas na base "EMG data for gestures Data Set" e assim identificar posições das maos. O link abaixo é referente a base de dados utilizada disponivel para download.
https://archive.ics.uci.edu/ml/datasets/EMG+data+for+gestures#

O codigo fonte esta disponivel publicamente no GitHub e pode ser acessado pelo link abaixo.
https://github.com/rmmenezes/ReconhecimentoDePadroes-UTFPR.git

Import das bibliotecas e estruturas necessárias

In [25]:
import os
import numpy as np

X = []
y = []
caracteristicas = []
classes = []

In [26]:
diretorio_db = "../data/EMG_data_for_gestures-master"
pastas = os.listdir(diretorio_db)
for pasta in pastas:
    arquivos = os.listdir(diretorio_db + '/' + pasta)
    #print(pasta)
    for arquivo in arquivos:
        #le o arquivo e divide linha por linha
        conteudo = open(diretorio_db + '/' + pasta + '/' + arquivo).read().split('\n')
        #remove ultima linha (linha em branco)
        conteudo.pop()
        #remove primeira linha (cabecalho)
        conteudo.pop(0)
        classe_atual = '0'
        experimento_caracteristicas = []
        experimento_classes = []
        for i in range(len(conteudo)):
            conteudo[i] = conteudo[i].split('\t')
            #remove primeira linha (campo "time")
            conteudo[i].pop(0)
            if conteudo[i][-1] != classe_atual:
                caracteristicas.append(experimento_caracteristicas)
                classes.append(experimento_classes)
                classe_atual = conteudo[i][-1]

                experimento_caracteristicas = []
                experimento_classes = []
                experimento_caracteristicas.append(conteudo[i][:-1])
                experimento_classes.append(conteudo[i][-1])
            else:
                experimento_caracteristicas.append(conteudo[i][:-1])
                experimento_classes.append(conteudo[i][-1])
            
        caracteristicas.append(experimento_caracteristicas)
        classes.append(experimento_classes)
        #print(len(caracteristicas))
        

for i in range(len(caracteristicas)):
    for j in range(len(caracteristicas[i])):
        caracteristicas[i][j] = np.array(caracteristicas[i][j])
    caracteristicas[i] = np.array(caracteristicas[i])
    classes[i] = np.array(classes[i])
caracteristicas = np.array(caracteristicas)
classes = np.array(classes)

print(caracteristicas.shape, classes.shape)
print(caracteristicas[0].shape)

(1816,) (1816,)
(1538, 8)


In [27]:
print(caracteristicas.shape, classes.shape)
print(caracteristicas[0].shape)
print(caracteristicas[1].shape)


x =np.array([[[1,2], [3,4]],
             [[5,6], [7,8]]])
print(x.shape)

print(np.sum(x, axis=0))
print('\n')
print(np.sum(x, axis=1))
print('\n')
print(np.sum(x, axis=2))

#resultado = np.sum(caracteristicas, axis=0)
#for i in range(len(resultado)):
#    resultado[i] /= len(caracteristicas)
#print(resultado.shape)




(1816,) (1816,)
(1538, 8)
(1494, 8)
(2, 2, 2)
[[ 6  8]
 [10 12]]


[[ 4  6]
 [12 14]]


[[ 3  7]
 [11 15]]


In [ ]:
quantos_valores = []

somas_geral = []
somas_temp = []

somas_quadraticas = []
somas_quadraticas_temp = []

somas_quadraticas_sem_modulo = []
somas_quadraticas_sem_modulo_temp = []

somas_wf = []
somas_wf_temp = []

for i in range(8):
    somas_temp.append(0.0)
    somas_quadraticas_temp.append(0.0)
    somas_wf_temp.append(0.0)
    somas_quadraticas_sem_modulo_temp.append(0.0)

for experimento in caracteristicas:
    for tipo in experimento:
        cont = 0
        for i in range(len(tipo)):
            somas_temp[i] += abs(float(tipo[i]))
            somas_quadraticas_temp[i] += abs(float(tipo[i])) ** 2
            somas_quadraticas_sem_modulo_temp[i] += float(tipo[i]) ** 2
            cont += 1
            if(cont < len(tipo) - 1):
                somas_wf_temp[i] = abs(float(tipo[i+1]) - float(tipo[i]))
    somas_geral.append(somas_temp)
    somas_quadraticas.append(somas_quadraticas_temp)
    somas_wf.append(somas_wf_temp)
    somas_quadraticas_sem_modulo.append(somas_quadraticas_sem_modulo_temp)
    
    #arrumar esses vetores, mover +1 nivel interno
    quantos_valores.append(cont)
    

print(len(somas_geral), len(somas_geral[0]))
print(len(somas_quadraticas), len(somas_quadraticas[0]))
print(len(somas_wf), len(somas_wf[0]))
print(len(somas_quadraticas_sem_modulo), len(somas_quadraticas_sem_modulo[0]))
print(len(quantos_valores))




In [8]:
import math

iemg = np.array(somas_geral)

mav = np.array(somas_geral)
for i in range(len(mav)):
    for j in range(len(mav[i])):
        mav[i][j] *= 1/len(quantos_valores)

ssi = np.array(somas_quadraticas)

mavs = mav
for i in range(len(mavs)-1):
    mavs[i] = mavs[i+1] - mavs[i]

var = np.array(somas_quadraticas_sem_modulo)
for i in range(len(var)):
    for j in range(len(var[i])):
        var[i][j] *= 1/(len(quantos_valores) - 1)

rms = ssi
for i in range(len(rms)):
    for j in range(len(rms[i])):
        rms[i][j] *= math.sqrt(1/len(quantos_valores))
        
wf = np.array(somas_wf)


print(iemg.shape)
print(mav.shape)
print(ssi.shape)
print(mavs.shape)
print(var.shape)
print(rms.shape)
print(wf.shape)
        


# [
#     [[1,2,3], [1,2,3], [1,2,3], [1,2,3]], # canal 1
#     [[1,2,3], [1,2,3], [1,2,3], [1,2,3]], # canal 2
#     [[1,2,3], [1,2,3], [1,2,3], [1,2,3]], # canal 3
# ]

# [
#     [
#         [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
#     ],
#     [
#         [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
#     ],
#     [
#         [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
#     ],
# ]



(1816, 8)
(1816, 8)
(1816, 8)
(1816, 8)
(1816, 8)
(1816, 8)
(1816, 8)


transoformação para o domínio da frequência

In [17]:
#from librosa import stft
#for i in range(len(caracteristicas)):
#    for j in range(len(caracteristicas[i])):
#        for k in range(len(caracteristicas[i][j])):
#            caracteristicas[i][j][k] = float(caracteristicas[i][j][k])
            
#for item in caracteristicas:
#    resultado = np.abs(stft(item, n_fft=512, hop_length=512))
#    print(resultado.shape)
#    input()

for item in caracteristica:
    resultado = np.apply_along_axis(item, 1, caracteristicas, n_fft=512, hop_length=512)
    print(resultado.shape)

AxisError: axis 2 is out of bounds for array of dimension 1